In [1]:
# Imports dependencies
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf 
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Importing the timesfm model (CPU version)
import timesfm

 See https://github.com/google-research/timesfm/blob/master/README.md for updated APIs.
Loaded PyTorch TimesFM, likely because python version is 3.11.11 (main, Mar  7 2025, 10:33:09) [Clang 16.0.0 (clang-1600.0.26.6)].


In [3]:
# Importing in the 200m parameter
tfm_one = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="cpu",
          per_core_batch_size=32,
          horizon_len=128,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-1.0-200m-pytorch"),
  )

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.58k [00:00<?, ?B/s]

torch_model.ckpt:   0%|          | 0.00/814M [00:00<?, ?B/s]

In [4]:
# Importing in the 500m parameter
tfm_two = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="cpu",
          per_core_batch_size=32,
          horizon_len=128,
          num_layers=50,
          use_positional_embedding=False,
          context_len=2048,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-2.0-500m-pytorch"),
  )

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [20]:
# Download BTC-USD data
btc = yf.download('BTC-USD', start='2022-01-01', end='2024-03-01')

# Save the closing price data to a CSV
btc[['Close']].to_csv("bitcoin_prices.csv")
btc_close = btc[['Close']].copy()

[*********************100%***********************]  1 of 1 completed


In [21]:
# Bitcoin database
btc.head()

Price,Close,High,Low,Open,Volume
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Date,,,,,
2022-01-01,47686.812500,47827.312500,46288.484375,46311.746094,24582667004
2022-01-02,47345.218750,47881.406250,46856.937500,47680.925781,27951569547
2022-01-03,46458.117188,47510.726562,45835.964844,47343.542969,33071628362
2022-01-04,45897.574219,47406.546875,45752.464844,46458.851562,42494677905
2022-01-05,43569.003906,46929.046875,42798.222656,45899.359375,36851084859


In [22]:
# Bitcoin closing price filtered database
btc_close.head()

Price,Close
Ticker,BTC-USD
Date,
2022-01-01,47686.812500
2022-01-02,47345.218750
2022-01-03,46458.117188
2022-01-04,45897.574219
2022-01-05,43569.003906


In [23]:
print(btc_close.shape)
print(btc.shape)

(790, 1)
(790, 5)


In [27]:
btc_close["unique_id"] = "btc"
btc_close.head

<bound method NDFrame.head of Price              Close unique_id
Ticker           BTC-USD          
Date                              
2022-01-01  47686.812500       btc
2022-01-02  47345.218750       btc
2022-01-03  46458.117188       btc
2022-01-04  45897.574219       btc
2022-01-05  43569.003906       btc
...                  ...       ...
2024-02-25  51733.238281       btc
2024-02-26  54522.402344       btc
2024-02-27  57085.371094       btc
2024-02-28  62504.789062       btc
2024-02-29  61198.382812       btc

[790 rows x 2 columns]>

In [11]:
# Select just the closing prices
prices = btc['Close'].values.reshape(-1, 1)

# Normalize prices (Min-Max scaling)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
prices_scaled = scaler.fit_transform(prices).flatten()

# Define context length
context_length = 512

# Ensure data is sufficient
if len(prices) <= context_length:
    raise ValueError("Data too short.")

# Prepare input tensor
input_tensor = torch.tensor(
    prices[-context_length:].reshape(1, context_length), dtype=torch.float32
)

In [24]:
forecast_horizon = 64

# Corrected forecast using the proper argument 'horizon'
forecast_tensor = tfm_one.forecast_on_df(
    input_tensor,
    freq = "D"
)

# Convert forecast tensor to numpy array
forecast = forecast_tensor.detach().cpu().numpy().reshape(-1, 1)
print(type(forecast))

# Inverse transform forecast to original scale
forecast_prices = scaler.inverse_transform(forecast).flatten()

AttributeError: 'Tensor' object has no attribute 'columns'